## Macromolecules : Binding Affinity

LAST REVISED: July 10, 2023

In [1]:
from src.modules.chemicals.chem_data import ChemData
from src.modules.reactions.reaction_dynamics import ReactionDynamics

In [2]:
# Initialize the system
chem = ChemData(names=["A", "B", "C"])

### Explore methods to manage the data structure for macromolecules

In [3]:
chem.add_macromolecules("M1")
chem.get_macromolecules()

['M1']

In [4]:
chem.set_binding_site_affinity("M1", 3, "A", 0.92)
chem.set_binding_site_affinity("M1", 8, "B", 1.1)
chem.set_binding_site_affinity("M1", 15, "A", 9.2)

chem.set_binding_site_affinity("M2", 1, "C", 5.6)
chem.set_binding_site_affinity("M2", 2, "A", 0.092)

In [5]:
chem.get_binding_sites("M1")

[3, 8, 15]

In [6]:
chem.get_binding_sites_and_ligands("M1")

{3: 'A', 8: 'B', 15: 'A'}

In [7]:
chem.get_binding_sites("M2")

[1, 2]

In [8]:
chem.get_binding_sites_and_ligands("M2")

{1: 'C', 2: 'A'}

In [9]:
chem.get_binding_site_affinity(macromolecule="M1", site_number=3)   # A "NamedTuple" gets returned

ChemicalAffinity(chemical='A', affinity=0.92)

In [10]:
aff = chem.get_binding_site_affinity(macromolecule="M1", site_number=8)
aff

ChemicalAffinity(chemical='B', affinity=1.1)

In [11]:
aff.chemical

'B'

In [12]:
aff.affinity

1.1

### Start setting up the dynamical system

In [13]:
dynamics = ReactionDynamics(chem_data=chem)

In [14]:
dynamics.set_macromolecules()      # By default, set counts to 1 for all the registered macromolecules

### Inspect some class attributes

In [15]:
dynamics.macro_system

{'M1': 1, 'M2': 1}

In [16]:
dynamics.macro_system_state

{'M1': {3: ('A', 0.0), 8: ('B', 0.0), 15: ('A', 0.0)},
 'M2': {1: ('C', 0.0), 2: ('A', 0.0)}}

In [17]:
dynamics.describe_state()

SYSTEM STATE at Time t = 0:
3 species:
  Species 0 (A). No concentrations set yet
  Species 1 (B). No concentrations set yet
  Species 2 (C). No concentrations set yet
Macro-molecules present, with their counts:  {'M1': 1, 'M2': 1}
Binding Occupancy fractions at the various binding sites for each of the macro-molecules:
     M1 || 3: 0.0 (A) | 8: 0.0 (B) | 15: 0.0 (A)
     M2 || 1: 0.0 (C) | 2: 0.0 (A)


### Set the initial concentrations of all the ligands

In [18]:
dynamics.set_conc(conc={"A": 9.2, "B": 0., "C": 0.56})
dynamics.describe_state()

SYSTEM STATE at Time t = 0:
3 species:
  Species 0 (A). Conc: 9.2
  Species 1 (B). Conc: 0.0
  Species 2 (C). Conc: 0.56
Macro-molecules present, with their counts:  {'M1': 1, 'M2': 1}
Binding Occupancy fractions at the various binding sites for each of the macro-molecules:
     M1 || 3: 0.0 (A) | 8: 0.0 (B) | 15: 0.0 (A)
     M2 || 1: 0.0 (C) | 2: 0.0 (A)


### Adjust the fractional occupancy of the various sites on the macromolecules, based on the current ligand concentrations

In [19]:
dynamics.update_occupancy()

In [20]:
dynamics.describe_state()

SYSTEM STATE at Time t = 0:
3 species:
  Species 0 (A). Conc: 9.2
  Species 1 (B). Conc: 0.0
  Species 2 (C). Conc: 0.56
Macro-molecules present, with their counts:  {'M1': 1, 'M2': 1}
Binding Occupancy fractions at the various binding sites for each of the macro-molecules:
     M1 || 3: 0.8999999930397401 (A) | 8: 4.434699570852307e-15 (B) | 15: 0.5 (A)
     M2 || 1: 0.10000000696026 (C) | 2: 0.9878048761855343 (A)


In [21]:
dynamics.chem_data.show_binding_affinities()        # Review the values the had given for the binding affinities

M1  :
   Site 3 - Binding affinity for A : 0.92
   Site 8 - Binding affinity for B : 1.1
   Site 15 - Binding affinity for A : 9.2
M2  :
   Site 1 - Binding affinity for C : 5.6
   Site 2 - Binding affinity for A : 0.092


#### Notes:
**[B] = 0** => Occupancy of binding site 8 on M1 is also zero

**[A] = 9.2** :   
            * 10x the binding affinity of A to site 3 on M1 (occupancy 0.9)  
            * same as the binding affinity of A to site 15 on M1 (occupancy 0.5)  
            * 100x the binding affinity of A to site 2 on M2 (occupancy almost 1, i.e. nearly saturated)
        
            
**[C] = 0.56** => 1/10 of the binding affinity of A to site 1 on M2 (occupancy 0.1)